Eleições 2022 - 2º Turno<br>
Dados apenas do estado de SP<Br>
<br>
Objetivos:<br>
- Importar e tratar o arquivo com o boletim da urna;<br>
- Descobrir as seções que cada candidato obteve mais votos que o outro e definir como ganhador daquela seção;<Br>
- Importar e tratar o arquivo com o perfil do eleitores;<br>
- Realizar a intersecção entre as seções que cada candidato venceu e o perfil do eleitor.<Br>

In [1]:
import os
import sys

In [2]:
os.environ["SPARK_PYTHON"] = sys.executable
os.environ["SPARK_DRIVER_PYTHON"] = sys.executable

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [4]:
spark = (
  SparkSession.builder
  .master('local')
  .appName('Eleicoes_2022')
  .getOrCreate()
)

### Boletim Urna 2022

In [5]:
# Carregar o arquivo com o boletim da urna
votos = spark.read.option("encoding", "ISO-8859-1").csv(
  "datasets/boletim_urna_2022.csv",
  inferSchema=True,
  header=True,
  sep=";"
  )

In [6]:
# Selecionar apenas determinadas colunas
votos = votos.select(
  "NM_MUNICIPIO", "NR_ZONA", "NR_SECAO", "DS_CARGO_PERGUNTA", "NM_VOTAVEL",
  "QT_VOTOS"
  )

In [7]:
# Renomear as colunas
votos = votos.withColumnsRenamed(
  {
    "NM_MUNICIPIO" : "Municipio",
    "NR_ZONA":"Zona",
    "NR_SECAO":"Secao",
    "DS_CARGO_PERGUNTA":"Cargo",
    "NM_VOTAVEL": "Candidato",
    "QT_VOTOS": "Votos"
  }
)

In [8]:
# Converter as colunas Zona, Seção e Numero para str
votos = votos.selectExpr(
  "cast(Municipio as string) Municipio",
  "cast(Zona as string) Zona",
  "cast(Secao as string) Secao",
  "cast(Cargo as string) Cargo",
  "cast(Candidato as string) Candidato",
  "cast(Votos as int) Votos"
)

In [9]:
# Capitalizar as Colunas em Maiúsculas
votos = (
  votos
  .withColumn("Candidato", initcap(col("Candidato")))
  .withColumn("Municipio", initcap(col("Municipio")))
  )

#### Presidente

In [10]:
# Obter apenas os valores para cargo de presidente
presidente = votos.filter(col("cargo") == "Presidente")

In [11]:
presidente = presidente.select("Municipio","Zona","Secao","Candidato","Votos")

In [12]:
presidente = (
  presidente
  .groupBy("Municipio","Zona", "Secao")
  .pivot("Candidato")
  .max("Votos")
  )

In [13]:
presidente = presidente.withColumnRenamed(
  "Jair Bolsonaro",
  "Bolsonaro"
)

In [15]:
presidente = presidente.withColumn(
  "Vencedor",
  when(
    col("Lula") > col("Bolsonaro"), "Lula"
  )
  .otherwise("Bolsonaro")
)

In [16]:
secoes_lula = presidente.filter(col("Vencedor") == "Lula")

In [17]:
secoes_lula = secoes_lula.select("Municipio", "Zona", "Secao")

In [18]:
secoes_bolsonaro = presidente.filter(col("Vencedor") == "Bolsonaro")

In [19]:
secoes_bolsonaro = secoes_bolsonaro.select("Municipio", "Zona", "Secao")

#### Governador

In [13]:
# Filtar votos apenas para cargo de governador
governador = (
  votos
  .filter(col("Cargo") == "Governador")
  .select("Municipio", "Zona", "Secao", "Candidato", "Votos")
  )

In [15]:
# Dinamizar colunas Candidatos e Votos
governador = (
  governador
  .groupBy("Municipio","Zona", "Secao")
  .pivot("Candidato")
  .max("Votos")
  )

In [18]:
# Simplificar nome do candidato Fernando Haddad para Haddad e remoção da
# acentuação no nome do candidato Tarcísio
governador = governador.withColumnsRenamed(
  {
    "Fernando Haddad":"Haddad",
    "Tarcísio": "Tarcisio"
  }
)

In [21]:
governador = governador.withColumn(
  "Vencedor",
  when(
    col("Haddad") > col("Tarcisio"), "Haddad"
  )
  .otherwise("Tarcisio")
)

In [25]:
secoes_governador = governador.select("Municipio", "Zona", "Secao", "Vencedor")

In [28]:
secoes_haddad = (
  secoes_governador
  .filter(col("Vencedor") == "Haddad")
  .drop(col("Vencedor"))
)

In [29]:
secoes_tarcisio = (
  secoes_governador
  .filter(col("Vencedor") == "Tarcisio")
  .drop(col("Vencedor"))
)

### Perfil Eleitor 2022

In [30]:
# Importar arquivo com perfil do eleitor
perfil = spark.read.option("encoding", "ISO-8859-1").csv(
  "datasets/perfil_eleitor_secao_2022_SP.csv",
  inferSchema=True,
  header=True,
  sep=";"
  )

In [31]:
# Selecionar colunas específicas
perfil = perfil.select(
  "NM_MUNICIPIO",
  "NR_ZONA",
  "NR_SECAO",
  "DS_GENERO",
  "DS_ESTADO_CIVIL",
  "DS_FAIXA_ETARIA",
  "DS_GRAU_ESCOLARIDADE",
  "QT_ELEITORES_PERFIL",
  "QT_ELEITORES_BIOMETRIA",
  "QT_ELEITORES_DEFICIENCIA",
  "QT_ELEITORES_INC_NM_SOCIAL"
)

In [32]:
# Renomear as colunas
perfil = perfil.withColumnsRenamed(
  {
    "NM_MUNICIPIO":"Municipio",
    "NR_ZONA":"Zona",
    "NR_SECAO":"Secao",
    "DS_GENERO":"Genero",
    "DS_ESTADO_CIVIL":"Estado Civil",
    "DS_FAIXA_ETARIA":"Faixa Etaria",
    "DS_GRAU_ESCOLARIDADE":"Escolaridade",
    "QT_ELEITORES_PERFIL":"Eleitores Geral",
    "QT_ELEITORES_BIOMETRIA":"Eleitores Biometria",
    "QT_ELEITORES_DEFICIENCIA":"Eleitores Deficiencia",
    "QT_ELEITORES_INC_NM_SOCIAL":"Eleitores Nome Social"
  }
)

In [33]:
# Capitalizar os valores das colunas
perfil = perfil.withColumns(
  {
    "Municipio":initcap(col("Municipio")),
    "Genero":initcap(col("Genero")),
    "Estado Civil":initcap(col("Estado Civil")),
    "Escolaridade":initcap(col("Escolaridade"))
  }
)

In [34]:
# Diminuir tamanho dos valores, não é necessário.
perfil = perfil.withColumn(
  "Escolaridade",
  when(
    col("Escolaridade") == "Ensino Médio Incompleto", "Médio Incompleto"
  )
  .when(
    col("Escolaridade") == "Ensino Médio Completo", "Médio Completo"
  )
  .when(
    col("Escolaridade") == "Ensino Fundamental Completo", "Fundamental Completo"
  )
  .when(
    col("Escolaridade") == "Ensino Fundamental Incompleto", "Fundamental Incompleto"
  )
  .when(
    col("Escolaridade") == "Ensino Médio Completo", "Médio Completo"
  )
  .when(
    col("Escolaridade") == "Lê E Escreve", "Lê e Escreve"
  )
  .when(
    col("Escolaridade") == "Analfabeto", "Analfabeto"
  )
  .when(
    col("Escolaridade") == "Superior Incompleto", "Superior Incompleto"
  )
  .when(
    col("Escolaridade") == "Superior Completo", "Superior Completo"
  )
  .otherwise(col("Escolaridade"))
)

In [35]:
# Há espaço em branco à direita na coluna 'Faixa Etaria'
perfil = (
  perfil
  .withColumn(
    "Faixa Etaria",
    rtrim(col("Faixa Etaria"))
    )
  )

#### Perfil Presidentes

In [26]:
perfil_lula = secoes_lula.join(
  perfil,
  [
    "Municipio",
    "Zona",
    "Secao"
  ],
  "inner",
)

In [51]:
perfil_bolsonaro = secoes_bolsonaro.join(
  perfil,
  [
    "Municipio",
    "Zona",
    "Secao"
  ],
  "inner"
)

#### Perfil Governadores

In [36]:
perfil_haddad = secoes_haddad.join(
  perfil,
  [
    "Municipio",
    "Zona",
    "Secao"
  ],
  "inner"
)

In [38]:
perfil_tarcisio = secoes_tarcisio.join(
  perfil,
  [
    "Municipio",
    "Zona",
    "Secao"
  ],
  "inner"
)